In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time

In [18]:
from utils import check_if_titles_match, lower_and_remove_diacritics

Already scraped
artist_name = "Severina"
artist_name = "Mia Dimšić"
artist_name = "Jelena Rozga"
artist_name = "Petar Grašo"
artist_name = "Đani Maršan"
artist_name = "Aerodrom"


In [32]:
artist_name = "Alen Vitasović"


In [33]:
df = pd.read_csv(f"data/{artist_name}_tekstovinet_lyricstranslate.csv")


In [34]:
df.head()

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,Album_lyricstranslate,Lyrics_lyricstranslate,EngLyrics_lyricstranslate,Url_lyricstranslate
0,H/8bQC7v_1_0,Alen Vitasović,Alen Vitasović,Ako tebi lazen ja,2417,"Mislim na tebe\r\nka iz spine curi vrime, brzo...","https://tekstovi.net/2,2448,42582.html",NaN,Ako tebi lažen ja,NaN,NaN,"Mislim na tebe\nka iz spine curi vrime, brzo g...",NaN,https://lyricstranslate.com/de/alen-vitasovi%C...
1,H/8bQC7v_1_1,Alen Vitasović,Alen Vitasović,Bura,7517,"Bura, svo je grane slomila\r\ni tvoje je tilo ...","https://tekstovi.net/2,2448,32922.html",NaN,Bura,NaN,NaN,"Bura, svo je grane slomila\ni tvoje je tilo ob...",NaN,https://lyricstranslate.com/en/alen-vitasovi%C...
2,H/8bQC7v_1_2,Alen Vitasović,Alen Vitasović,Ca mi sad ne smi rec (feat. Zdenko Hrsak),1705,"Vani je cvika, po krovu ledi tuce \r\nnema kuc...","https://tekstovi.net/2,2448,52349.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H/8bQC7v_1_3,Alen Vitasović,Alen Vitasović,Cesta priko mora,5463,Ja san s tobon biti stija\r\n i po svitu san h...,"https://tekstovi.net/2,2448,47629.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H/8bQC7v_1_4,Alen Vitasović,Alen Vitasović,Cvit,1052,Moglo je pasati cuda bolje\r\nvrimena je bilo ...,"https://tekstovi.net/2,2448,58376.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df["Lyrics_lyricstranslate"].isnull().sum()

12

In [23]:
#df[df["Lyrics_lyricstranslate"].isnull()]

In [24]:
def scrape_for_authors_and_albums(song_name, artist_name):
    #search
    driver.find_element(By.CLASS_NAME, 'search-page').clear()
    search_box = driver.find_element(By.CLASS_NAME, 'search-page')
    song_name = lower_and_remove_diacritics(song_name)
    search_box.send_keys(song_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)
    
    
    #get results of search
    #search_result = driver.find_element(By.CLASS_NAME, 'search-result')
    try:
        song_results = driver.find_element(By.XPATH, f"//*[contains(text(), 'Pjesme')]/following-sibling::ul")
        song_results = song_results.find_elements(By.TAG_NAME, 'a')

        #go through search results
        for song in song_results:

            song.send_keys(Keys.CONTROL + Keys.RETURN)
            window_handles = driver.window_handles

            driver.switch_to.window(driver.window_handles[1])
            time.sleep(1)            

            if  check_performer(artist_name):
                subfields, authors = get_authors()
                albums = get_albums()
                song_url = driver.current_url
                song_name_website = driver.find_element(By.XPATH, f"//*[contains(text(), 'Pjesma')]/following-sibling::h1").text
                driver.close()
                driver.switch_to.window(window_name=window_handles[0])
                return subfields, authors, albums, song_url, song_name_website
                
            else:
                driver.close()
                driver.switch_to.window(window_name=window_handles[0])
    
    except Exception as e: print(f"Problem with scraping authors and albums for song {song_name}")

    return False    


In [25]:
def check_performer(artist_name):
    authors = driver.find_elements(By.TAG_NAME, "h3")[1:]
    artist_name = lower_and_remove_diacritics(artist_name)
    for index, author in enumerate(authors):
        subfield = author.text.capitalize()
        if subfield == "Izvedba":
            siblings_of_current = driver.find_elements(By.XPATH, f"//*[contains(text(), '{subfield}')]/following-sibling::a")
            if index<len(authors)-1: 
                siblings_of_next = driver.find_elements(By.XPATH, f"//*[contains(text(), '{authors[index+1].text.capitalize()}')]/following-sibling::a")
                result = [r.text for r in siblings_of_current[0: len(siblings_of_current)-len(siblings_of_next)]]
            else: result = [r.text for r in siblings_of_current] 
            
    return True if len([r for r in result if artist_name in lower_and_remove_diacritics(r)])>0 else False


In [26]:
def get_authors():
    all_fields = []
    all_authors = []
    #authors = driver.find_elements(By.TAG_NAME, "h3")[1:]
    authors = driver.find_elements(By.XPATH, f"//*[contains(text(), 'Autori')]/following-sibling::h3")
    for index, author in enumerate(authors):
        subfield = author.text.capitalize()
        siblings_of_current = driver.find_elements(By.XPATH, f"//*[contains(text(), '{subfield}')]/following-sibling::a")
        if len(siblings_of_current)>0:
            if index<len(authors)-1: 
                siblings_of_next = driver.find_elements(By.XPATH, f"//*[contains(text(), '{authors[index+1].text.capitalize()}')]/following-sibling::a")
                result = [r.text for r in siblings_of_current[0: len(siblings_of_current)-len(siblings_of_next)]]
            else: result = [r.text for r in siblings_of_current]
            all_fields.append(subfield)
            all_authors.append(str(result))
            
    return all_fields, all_authors


In [27]:
def get_albums():
    albums = driver.find_elements(By.XPATH, f"//*[contains(text(), 'Albumi')]/following-sibling::div")
    #return [f"{album.text}-{album.find_element(By.TAG_NAME, 'a').text}" for album in albums]
    return [{album.text} for album in albums]      
    

In [35]:
ser = Service("chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://diskografija.com/")


df_new = df.copy()#[0:5]
for row in df_new.iterrows():
    artist_name = row[1].Artist_diskografija
    song_name = row[1].Song_lyricstranslate
    if type(song_name)==float: song_name = row[1].Song_tekstovinet
    try: song_name = song_name[0:song_name.index("(")]
    except:pass
    
    try: 
        subfields, authors, albums, song_url, song_title_website = scrape_for_authors_and_albums(song_name, artist_name)
    
        for subfield, author in zip(subfields, authors):
            df.loc[row[0], f"{subfield}_diskografija"] = author
            
        for i, album in enumerate(albums):
            df.loc[row[0], f"Album_{i+1}_diskografija"] = album
        df.loc[row[0], "Url_diskografija"] = song_url
        df.loc[row[0], "Song_diskografija"] = song_title_website

        print(f"Succesfully found information for {song_name} from {artist_name}")
    
    except Exception as e:
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


Succesfully found information for Ako tebi lažen ja from Alen Vitasović
#######
Succesfully found information for Bura from Alen Vitasović
#######
Succesfully found information for Ca mi sad ne smi rec  from Alen Vitasović
#######
Succesfully found information for Cesta priko mora from Alen Vitasović
#######
Problem with finding info for Cvit from Alen Vitasović
cannot unpack non-iterable bool object
#######
Succesfully found information for Dajla from Alen Vitasović
#######
Succesfully found information for Groznjana grada  from Alen Vitasović
#######
Succesfully found information for Gušti su gušti from Alen Vitasović
#######
Succesfully found information for Ja ne gren from Alen Vitasović
#######
Problem with finding info for Ja san Istrijan from Alen Vitasović
cannot unpack non-iterable bool object
#######
Succesfully found information for Jenu noć from Alen Vitasović
#######
Problem with scraping authors and albums for song kad ce vrime od kruha i grozja
Problem with finding info 

In [36]:
check_if_titles_match(df, "Song_tekstovinet", "Song_diskografija")

,Song_tekstovinet,Song_diskografija


In [37]:
df["Lyrics_lyricstranslate"].isnull().sum()

20

In [38]:
df.to_csv(f"data/{artist_name}_tekstovinet_lyricstranslate_diskografija.csv", index=False)
